In [7]:
import os
import numpy as np
import pandas as pd
from reconciliation_utils import *

In [ ]:
estimator = "DeepAR"

ids = np.empty((0, ))

labels = np.empty((0, 1941))
forecasts = np.empty((0, 1941))

for level in range(1, 13):
    level_dir = f"../result/level {level}"
    for estimator_root, estimator_dirs, estimator_files in os.walk(level_dir):
        for estimator_dir in estimator_dirs:
            if estimator_dir.startswith(estimator):
                estimator_dir = os.path.join(estimator_root, estimator_dir)
                for estimator_file in os.listdir(estimator_dir):
                    if estimator_file.startswith('train_labels'):
                        with open(os.path.join(estimator_dir, estimator_file), 'rb') as pickle_file:
                            datas = pd.read_pickle(pickle_file)
                            arrays = np.array(datas)
                            arrays = np.squeeze(arrays, axis=-1)
                            arrays = arrays[:, -28:]
                            train_labels = np.concatenate((train_labels, arrays), axis=0)
                    if estimator_file.startswith('valid_labels'):
                        with open(os.path.join(estimator_dir, estimator_file), 'rb') as pickle_file:
                            datas = pd.read_pickle(pickle_file)
                            arrays = np.array(datas)
                            arrays = np.squeeze(arrays, axis=-1)
                            arrays = arrays[:, -28:]
                            valid_labels = np.concatenate((valid_labels, arrays), axis=0)
                    if estimator_file.startswith('test_labels'):
                        with open(os.path.join(estimator_dir, estimator_file), 'rb') as pickle_file:
                            datas = pd.read_pickle(pickle_file)
                            arrays = np.array(datas)
                            arrays = np.squeeze(arrays, axis=-1)
                            arrays = arrays[:, -28:]
                            test_labels = np.concatenate((test_labels, arrays), axis=0)
                    if estimator_file.startswith('train_forecasts'):
                        with open(os.path.join(estimator_dir, estimator_file), 'rb') as pickle_file:
                            datas = pd.read_pickle(pickle_file)
                            print(datas)
                            ids = np.concatenate((ids, np.array([data.item_id for data in datas])), axis=0)
                            arrays = np.array([data.quantile(0.5) for data in datas])
                            print(arrays.shape)
                            train_forecasts = np.concatenate((train_forecasts, arrays), axis=0)
                    if estimator_file.startswith('valid_forecasts'):
                        with open(os.path.join(estimator_dir, estimator_file), 'rb') as pickle_file:
                            datas = pd.read_pickle(pickle_file)
                            arrays = np.array([data.quantile(0.5) for data in datas])
                            valid_forecasts = np.concatenate((valid_forecasts, arrays), axis=0)
                    if estimator_file.startswith('test_forecasts'):
                        with open(os.path.join(estimator_dir, estimator_file), 'rb') as pickle_file:
                            datas = pd.read_pickle(pickle_file)
                            arrays = np.array([data.quantile(0.5) for data in datas])
                            test_forecasts = np.concatenate((test_forecasts, arrays), axis=0)

In [ ]:
node_mapping = {id: idx for idx, id in enumerate(ids)}
node_info = {id: parse_id_components(id) for id in ids}

In [ ]:
edges, edge_types = create_edges(node_mapping)
   
# 4. 결과 저장
# np.save('edges.npy', edges)
# np.save('edge_types.npy', edge_types)
   
# with open('node_mapping.pkl', 'wb') as f:
#     pickle.dump(node_mapping, f)
   
# 5. 통계 출력
print(f"Total nodes: {len(node_mapping)}")
print(f"Total edges: {len(edges)}")
print(f"Up edges: {np.sum(edge_types == 0)}")
print(f"Down edges: {np.sum(edge_types == 1)}")
print(f"Cross edges: {np.sum(edge_types == 2)}")

In [ ]:
import torch
import numpy as np

device='cuda' if torch.cuda.is_available() else 'cpu'

def numpy_to_tensor(forecasts, edges, edge_types, device=device):
    if isinstance(forecasts, np.ndarray):
        forecasts_tensor = torch.FloatTensor(forecasts).to(device)
    elif isinstance(forecasts, torch.Tensor):
        forecasts_tensor = forecasts.float().to(device)
        
    if isinstance(edges, np.ndarray):
        edges_tensor = torch.LongTensor(edges).to(device)
    elif isinstance(edges, torch.Tensor):
        edges_tensor = edges.long().to(device)
        
    if isinstance(edge_types, np.ndarray):
        edge_types_tensor = torch.LongTensor(edge_types).to(device)
    elif isinstance(edge_types, torch.Tensor):
        edge_types_tensor = edge_types.long().to(device)
        
    return forecasts_tensor, edges_tensor, edge_types_tensor

def create_train_val_split(forecasts, targets, edges, edge_types, val_ratio=0.2, random_state=42):
    """
    학습 및 검증 데이터셋 분할
    
    Args:
        forecasts: 예측값 텐서
        targets: 실제값 텐서 
        edges: 엣지 텐서
        edge_types: 엣지 타입 텐서
        val_ratio: 검증 세트 비율
        random_state: 랜덤 시드
        
    Returns:
        train_forecasts, val_forecasts: 학습/검증용 예측값
        train_targets, val_targets: 학습/검증용 실제값
    """
    # 재현성을 위한 시드 설정
    np.random.seed(random_state)
    torch.manual_seed(random_state)
    
    # 노드 수
    num_nodes = forecasts.shape[0]
    
    # 노드 인덱스 섞기
    indices = np.random.permutation(num_nodes)
    split_idx = int(num_nodes * (1 - val_ratio))
    
    train_indices = indices[:split_idx]
    val_indices = indices[split_idx:]
    
    # 학습/검증 데이터 분할
    train_forecasts = forecasts[train_indices]
    val_forecasts = forecasts[val_indices]
    
    train_targets = targets[train_indices]
    val_targets = targets[val_indices]
    
    return train_forecasts, val_forecasts, train_targets, val_targets, edges, edge_types

# 사용 예시:
"""
# 데이터 준비
forecasts_tensor, edges_tensor, edge_types_tensor = prepare_data_for_gnn(
    forecasts=test_forecasts,
    edges=edges,
    edge_types=edge_types
)

# 학습/검증 분할
train_forecasts, val_forecasts, train_targets, val_targets, edges, edge_types = create_train_val_split(
    forecasts=forecasts_tensor,
    targets=targets_tensor,
    edges=edges_tensor,
    edge_types=edge_types_tensor
)

# 모델 학습
model = GATModel(input_dim=train_forecasts.shape[1])
trainer = Trainer(model)
history = trainer.train(
    train_forecasts, train_targets,
    val_forecasts, val_targets,
    edges, edge_types
)
"""

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import mean_absolute_error, mean_squared_error

def prepare_time_based_data(forecasts, targets, time_splits=[1885, 1913, 1941]):
    """
    시간 단위로 데이터 분할
    
    Args:
        forecasts: 전체 예측값 배열 [num_nodes, 1941]
        targets: 전체 실제값 배열 [num_nodes, 1941]
        time_splits: 시간 분할 지점 (예: [1885, 1913, 1941])
    
    Returns:
        train_forecasts: 학습용 예측값
        train_targets: 학습용 실제값
        val_forecasts: 검증용 예측값
        val_targets: 검증용 실제값
        test_forecasts: 테스트용 예측값 (최종 예측 타겟)
    """
    # 시간 분할 지점
    train_end = time_splits[0]
    val_end = time_splits[1]
    
    # 학습 데이터: 1 ~ train_end 일
    train_forecasts = forecasts[:, :train_end]
    train_targets = targets[:, :train_end]
    
    # 검증 데이터: train_end+1 ~ val_end 일
    val_forecasts = forecasts[:, train_end:val_end]
    val_targets = targets[:, train_end:val_end]
    
    # 테스트 데이터: val_end+1 ~ 끝
    test_forecasts = forecasts[:, val_end:]
    
    return train_forecasts, train_targets, val_forecasts, val_targets, test_forecasts

def prepare_gnn_data(train_forecasts, train_targets, val_forecasts, val_targets, 
                     test_forecasts, edges, edge_types, device):
    """데이터를 PyTorch Tensor로 변환"""
    # 학습/검증 데이터
    train_forecasts_tensor = torch.FloatTensor(train_forecasts).to(device)
    train_targets_tensor = torch.FloatTensor(train_targets).to(device)
    
    val_forecasts_tensor = torch.FloatTensor(val_forecasts).to(device)
    val_targets_tensor = torch.FloatTensor(val_targets).to(device)
    
    # 테스트 데이터
    test_forecasts_tensor = torch.FloatTensor(test_forecasts).to(device)
    
    # 그래프 구조
    edges_tensor = torch.LongTensor(edges).to(device)
    edge_types_tensor = torch.LongTensor(edge_types).to(device)
    
    return (train_forecasts_tensor, train_targets_tensor, 
            val_forecasts_tensor, val_targets_tensor, 
            test_forecasts_tensor, edges_tensor, edge_types_tensor)

class GATModel(torch.nn.Module):
    # 기존에 정의한 GATModel 클래스 코드
    pass  # 실제 코드에서는 GATModel 전체 정의

class Trainer:
    def __init__(
        self,
        model,
        learning_rate=1e-3,
        weight_decay=1e-5,
        device='cuda' if torch.cuda.is_available() else 'cpu'
    ):
        self.model = model.to(device)
        self.device = device
        self.optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        self.scheduler = ReduceLROnPlateau(self.optimizer, mode='min', patience=5, factor=0.5)
        
    def train_epoch(self, forecasts, edges, edge_types, targets):
        self.model.train()
        self.optimizer.zero_grad()
        
        # Forward pass
        predictions = self.model(forecasts, edges, edge_types)
        
        # Compute loss
        loss = torch.nn.functional.mse_loss(predictions, targets)
        
        # Backward pass
        loss.backward()
        self.optimizer.step()
        
        return loss.item()
    
    def validate(self, forecasts, edges, edge_types, targets):
        self.model.eval()
        with torch.no_grad():
            predictions = self.model(forecasts, edges, edge_types)
            val_loss = torch.nn.functional.mse_loss(predictions, targets).item()
            
            # 추가 메트릭
            mae = mean_absolute_error(targets.cpu().numpy(), predictions.cpu().numpy())
            rmse = np.sqrt(mean_squared_error(targets.cpu().numpy(), predictions.cpu().numpy()))
            
        return val_loss, mae, rmse
    
    def train(
        self,
        train_forecasts,
        train_targets,
        val_forecasts,
        val_targets,
        edges,
        edge_types,
        epochs=100,
        patience=10
    ):
        best_val_loss = float('inf')
        patience_counter = 0
        history = {'train_loss': [], 'val_loss': [], 'val_mae': [], 'val_rmse': []}
        
        for epoch in range(epochs):
            # 학습
            train_loss = self.train_epoch(train_forecasts, edges, edge_types, train_targets)
            
            # 검증
            val_loss, val_mae, val_rmse = self.validate(val_forecasts, edges, edge_types, val_targets)
            
            # 학습률 조정
            self.scheduler.step(val_loss)
            
            # 메트릭 기록
            history['train_loss'].append(train_loss)
            history['val_loss'].append(val_loss)
            history['val_mae'].append(val_mae)
            history['val_rmse'].append(val_rmse)
            
            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                # 최고 모델 저장
                torch.save(self.model.state_dict(), 'best_gnn_model.pt')
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f"Early stopping at epoch {epoch}")
                    break
            
            if epoch % 10 == 0:
                print(f"Epoch {epoch}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}, "
                      f"val_mae={val_mae:.4f}, val_rmse={val_rmse:.4f}")
        
        return history

def run_gnn_experiment(time_splits=[1885, 1913, 1941]):
    """GNN 실험 실행"""
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")
    
    # 1. 데이터 로드
    try:
        forecasts = np.load('all_forecasts.npy')  # 모든 노드의 전체 기간 예측값
        targets = np.load('all_targets.npy')     # 모든 노드의 전체 기간 실제값
        
        # 그래프 구조 로드
        edges = np.load('edges.npy')
        edge_types = np.load('edge_types.npy')
        
        print(f"Loaded data - Forecasts: {forecasts.shape}, Targets: {targets.shape}")
        print(f"Graph structure - Edges: {edges.shape}, Edge types: {edge_types.shape}")
        
    except FileNotFoundError:
        print("Required data files not found!")
        return
    
    # 2. 시간 기반 데이터 분할
    print(f"Splitting data based on time points: {time_splits}")
    train_forecasts, train_targets, val_forecasts, val_targets, test_forecasts = prepare_time_based_data(
        forecasts, targets, time_splits
    )
    
    print(f"Split shapes - Train: {train_forecasts.shape}, Val: {val_forecasts.shape}, Test: {test_forecasts.shape}")
    
    # 3. PyTorch Tensor 변환
    train_forecasts_tensor, train_targets_tensor, \
    val_forecasts_tensor, val_targets_tensor, \
    test_forecasts_tensor, edges_tensor, edge_types_tensor = prepare_gnn_data(
        train_forecasts, train_targets, val_forecasts, val_targets, 
        test_forecasts, edges, edge_types, device
    )
    
    # 4. GNN 모델 초기화
    # 각 시계열의 길이를 입력 차원으로 설정
    input_dim = train_forecasts.shape[1]  # 학습 기간의 길이
    print(f"Initializing GNN model with input_dim={input_dim}")
    
    model = GATModel(input_dim=input_dim, hidden_dim=256, num_layers=3)
    trainer = Trainer(model, device=device)
    
    # 5. 모델 학습
    print("Training GNN model...")
    history = trainer.train(
        train_forecasts_tensor, train_targets_tensor,
        val_forecasts_tensor, val_targets_tensor,
        edges_tensor, edge_types_tensor,
        epochs=100,
        patience=10
    )
    
    # 6. 검증 데이터에서의 성능 평가
    print("Evaluating on validation data...")
    model.load_state_dict(torch.load('best_gnn_model.pt'))
    _, val_mae, val_rmse = trainer.validate(
        val_forecasts_tensor, edges_tensor, edge_types_tensor, val_targets_tensor
    )
    
    print(f"Validation performance - MAE: {val_mae:.4f}, RMSE: {val_rmse:.4f}")
    
    # 7. 테스트 데이터(1914~1941일)에 적용하여 예측값 보정
    print("Applying GNN to adjust test forecasts...")
    model.eval()
    with torch.no_grad():
        adjusted_forecasts = model(test_forecasts_tensor, edges_tensor, edge_types_tensor)
    
    # 8. 결과 저장
    adjusted_forecasts_np = adjusted_forecasts.cpu().numpy()
    np.save('adjusted_test_forecasts.npy', adjusted_forecasts_np)
    
    # 9. 학습 히스토리 저장
    with open('train_history.pkl', 'wb') as f:
        pickle.dump(history, f)
    
    print("Experiment completed successfully!")
    return history, adjusted_forecasts_np

def analyze_results(original_forecasts, adjusted_forecasts, targets=None, num_samples=5):
    """결과 분석 및 시각화"""
    # 랜덤 샘플 선택
    np.random.seed(42)
    sample_indices = np.random.choice(original_forecasts.shape[0], num_samples, replace=False)
    
    # 원본 예측값과 조정된 예측값 비교
    plt.figure(figsize=(15, num_samples*4))
    
    for i, idx in enumerate(sample_indices):
        plt.subplot(num_samples, 1, i+1)
        plt.plot(original_forecasts[idx], 'b-', label='Original Forecast')
        plt.plot(adjusted_forecasts[idx], 'r-', label='GNN Adjusted')
        
        if targets is not None:
            plt.plot(targets[idx], 'g--', label='Actual')
            
            # 오차 계산
            orig_rmse = np.sqrt(mean_squared_error(targets[idx], original_forecasts[idx]))
            adj_rmse = np.sqrt(mean_squared_error(targets[idx], adjusted_forecasts[idx]))
            improvement = (orig_rmse - adj_rmse) / orig_rmse * 100
            
            plt.title(f'Node {idx} - Improvement: {improvement:.2f}% (RMSE: {orig_rmse:.4f} → {adj_rmse:.4f})')
        else:
            plt.title(f'Node {idx} - Original vs GNN Adjusted')
            
        plt.legend()
    
    plt.tight_layout()
    plt.savefig('forecast_comparison.png', dpi=300)
    plt.show()
    
    # 전체 성능 개선 분석 (타겟 데이터가 있는 경우)
    if targets is not None:
        orig_rmse = np.sqrt(mean_squared_error(targets.flatten(), original_forecasts.flatten()))
        adj_rmse = np.sqrt(mean_squared_error(targets.flatten(), adjusted_forecasts.flatten()))
        improvement = (orig_rmse - adj_rmse) / orig_rmse * 100
        
        print(f"Overall performance:")
        print(f"Original RMSE: {orig_rmse:.4f}")
        print(f"Adjusted RMSE: {adj_rmse:.4f}")
        print(f"Improvement: {improvement:.2f}%")
        
        return orig_rmse, adj_rmse, improvement

# 메인 실행 함수
def main():
    # 시간 분할 지점 설정
    time_splits = [1885, 1913, 1941]  # 학습 종료, 검증 종료, 테스트 종료
    
    # GNN 실험 실행
    history, adjusted_forecasts = run_gnn_experiment(time_splits)
    
    # 테스트 결과 로드
    test_forecasts = np.load('all_forecasts.npy')[:, time_splits[1]:]  # 원본 예측값
    test_targets = np.load('all_targets.npy')[:, time_splits[1]:]      # 실제값
    
    # 결과 분석
    analyze_results(test_forecasts, adjusted_forecasts, test_targets)

if __name__ == "__main__":
    main()

In [ ]:
len(test_forecasts)

In [ ]:
import numpy as np
import pandas as pd


levels = [1, 3, 10, 3, 7, 9, 21, 30, 70, 3049, 9147, 30490]

start_idx = 0
for i, level in enumerate(levels, start=1):
    end_idx = start_idx + level
    
    test_id_level = test_ids[start_idx:end_idx]
    df = pd.DataFrame(test_id_level, columns=['test_id'])
    df.to_csv(f'test_id_level_{i}.csv', index=False)
    
    start_idx = end_idx

In [ ]:
def calculate_wrmsse(y_true, y_pred):
    sales = pd.read_csv('../data/original/sales_train_validation.csv')
    sell_prices = pd.read_csv('../data/original/sell_prices.csv')

    sales = sales.iloc[:, 6:].values

    sell_prices['id'] = sell_prices['store_id'] + '_' + sell_prices['item_id']
    sell_prices = sell_prices[sell_prices['wm_yr_wk'] <= 11613]
    sell_prices = sell_prices.pivot(index='id', columns='wm_yr_wk', values='sell_price')
    sell_prices = sell_prices.values

    N, h = y_true.shape 
    w = sell_prices.shape[1]  

    daily_prices = np.repeat(sell_prices, repeats=7, axis=1)[:, -sales.shape[1]:]
    daily_prices = np.where(np.isnan(daily_prices), np.nan, daily_prices)
    
    squared_errors = np.mean((y_true - y_pred) ** 2, axis=1)
    scale = np.mean(np.diff(sales, axis=1) ** 2, axis=1)
    rmsse = np.sqrt(squared_errors / (scale + 1e-10))

    total_revenue = np.nansum(sales[:, -28:] * daily_prices[:, -28:], axis=1) 
    weight = total_revenue / np.nansum(total_revenue) 

    wrmsse = np.nansum(weight * rmsse)
    
    return wrmsse

In [ ]:
calculate_wrmsse(test_labels, test_forecasts) # 12

In [ ]:
from tqdm import tqdm
from scipy.linalg import pinv

def create_S(y_id):
    sales = pd.read_csv('../data/original/sales_train_validation.csv')
    sales['id'] = sales['id'].str.replace('_validation', '') # 30490

    states = sales['state_id'].unique()  
    stores = sales['store_id'].unique()
    cats = sales['cat_id'].unique()
    depts = sales['dept_id'].unique()
    states_cats = [f"{state}_{cat}" for state in states for cat in cats]
    states_depts = [f"{state}_{dept}" for state in states for dept in depts]
    stores_cats = [f"{store}_{cat}" for store in stores for cat in cats]
    stores_depts = [f"{store}_{dept}" for store in stores for dept in depts]
    items = sales['item_id'].unique()
    items_states = [f"{item}_{state}" for item in items for state in states]
    items_stores = [f"{item}_{store}" for item in items for store in stores]

    S = np.zeros((42840, 30490))

    for i, id in tqdm(enumerate(y_id), total=len(y_id)):
        if id == 'total':
            S[0, :] = 1            
        elif id in states:
            S[i, :] = sales['id'].isin(sales[sales['state_id'] == id]['id']).astype(int).values
        elif id in stores:
            S[i, :] = sales['id'].isin(sales[sales['store_id'] == id]['id']).astype(int).values
        elif id in cats:
            S[i, :] = sales['id'].isin(sales[sales['cat_id'] == id]['id']).astype(int).values
        elif id in depts:
            S[i, :] = sales['id'].isin(sales[sales['dept_id'] == id]['id']).astype(int).values
        elif id in states_cats:
            state, cat = id.split('_')
            S[i, :] = sales['id'].isin(sales[(sales['state_id'] == state) & (sales['cat_id'] == cat)]['id']).astype(int).values
        elif id in states_depts:
            splitted_id = id.split('_')
            state, dept = splitted_id[0], '_'.join(splitted_id[1:])
            S[i, :] = sales['id'].isin(sales[(sales['state_id'] == state) & (sales['dept_id'] == dept)]['id']).astype(int).values
        elif id in stores_cats:
            splitted_id = id.split('_')
            store, cat = '_'.join(splitted_id[:2]), splitted_id[2]
            S[i, :] = sales['id'].isin(sales[(sales['store_id'] == store) & (sales['cat_id'] == cat)]['id']).astype(int).values
        elif id in stores_depts:
            splitted_id = id.split('_')
            store, dept = '_'.join(splitted_id[:2]), '_'.join(splitted_id[2:])
            S[i, :] = sales['id'].isin(sales[(sales['store_id'] == store) & (sales['dept_id'] == dept)]['id']).astype(int).values
        elif id in items:
            S[i, :] = sales['id'].isin(sales[sales['item_id'] == id]['id']).astype(int).values
        elif id in items_states:
            splitted_id = id.split('_')
            item, state = '_'.join(splitted_id[:3]), '_'.join(splitted_id[3:])
            S[i, :] = sales['id'].isin(sales[(sales['item_id'] == item) & (sales['state_id'] == state)]['id']).astype(int).values
        elif id in items_stores:
            splitted_id = id.split('_')
            item, store = '_'.join(splitted_id[:3]), '_'.join(splitted_id[3:])
            S[i, :] = sales['id'].isin(sales[(sales['item_id'] == item) & (sales['store_id'] == store)]['id']).astype(int).values
        else:
            print(f"Error: {id} not found")

    return S

def compute_W(y_actual, y_pred):
    E = y_actual - y_pred
    W = (1 / (E.shape[1] - 1)) * (E @ E.T)
    return W

S = create_S(test_ids)
W = compute_W(test_labels, test_forecasts)

In [ ]:
from scipy.linalg import inv

def convert_to_appropriate_dtype(matrix):
    c_min = matrix.min()
    c_max = matrix.max()
    
    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
        return matrix.astype(np.float16)
    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
        return matrix.astype(np.float32)
    else:
        return matrix.astype(np.float64)

S = convert_to_appropriate_dtype(S)
W = convert_to_appropriate_dtype(W)
test_forecasts = convert_to_appropriate_dtype(test_forecasts)

W_inv = inv(W)
ST_Winv = S.T @ W_inv
ST_Winv_S = ST_Winv @ S
ST_Winv_S_inv = inv(ST_Winv_S)
S_ST_Winv_S_inv = S @ ST_Winv_S_inv

In [ ]:
import os
import numpy as np
from joblib import Parallel, delayed

def chunked_dot_memmap(A, B, chunk_size, n_jobs=-1, temp_dir='C:/temp'):
    def process_chunk_memmap(start, end, A, B, result_memmap):
        result_memmap[start:end] = A[start:end] @ B

    n_chunks = (A.shape[0] + chunk_size - 1) // chunk_size
    result_shape = (A.shape[0], B.shape[1])
    
    # Ensure the temporary directory exists
    os.makedirs(temp_dir, exist_ok=True)
    
    # Use an absolute path for the memory-mapped file
    result_memmap_path = os.path.join(temp_dir, 'result_memmap.dat')
    result_memmap = np.memmap(result_memmap_path, dtype=A.dtype, mode='w+', shape=result_shape)

    Parallel(n_jobs=n_jobs)(
        delayed(process_chunk_memmap)(i * chunk_size, min((i + 1) * chunk_size, A.shape[0]), A, B, result_memmap)
        for i in range(n_chunks)
    )

    return result_memmap

# Example usage
# Assuming S_ST_Winv_S_inv and ST_Winv are already defined
chunk_size = 1000  # Adjust chunk size based on available memory
P = chunked_dot_memmap(S_ST_Winv_S_inv, ST_Winv, chunk_size=chunk_size)

# Use the result
print("Shape of P:", P.shape)

In [ ]:
test_forecasts_reconciled = P @ test_forecasts

print("Before reconciliation:", test_forecasts[:10])
print("After reconciliation:", test_forecasts_reconciled[:10])

In [ ]:
def mint_reconciliation(S, W, y_pred):
    W_inv = inv(W)
    P = S @ inv(S.T @ W_inv @ S) @ S @ W_inv
    return P @ y_pred

test_forecasts_reconciled = mint_reconciliation(S, W, test_forecasts)

print("Before reconcilation:", test_forecasts[:10])
print("After reconcilation:", test_forecasts_reconciled[:10])

In [ ]:
def calculate_wrmsse(y_true, y_pred):
    sales = pd.read_csv('../data/original/sales_train_validation.csv')
    sell_prices = pd.read_csv('../data/original/sell_prices.csv')

    sales = sales.iloc[:, 6:].values

    sell_prices['id'] = sell_prices['store_id'] + '_' + sell_prices['item_id']
    sell_prices = sell_prices[sell_prices['wm_yr_wk'] <= 11613]
    sell_prices = sell_prices.pivot(index='id', columns='wm_yr_wk', values='sell_price')
    sell_prices = sell_prices.values

    N, h = y_true.shape 
    w = sell_prices.shape[1]  

    daily_prices = np.repeat(sell_prices, repeats=7, axis=1)[:, -sales.shape[1]:]
    daily_prices = np.where(np.isnan(daily_prices), np.nan, daily_prices)
    
    squared_errors = np.mean((y_true - y_pred) ** 2, axis=1)
    scale = np.mean(np.diff(sales, axis=1) ** 2, axis=1)
    rmsse = np.sqrt(squared_errors / (scale + 1e-10))

    total_revenue = np.nansum(sales[:, -28:] * daily_prices[:, -28:], axis=1) 
    weight = total_revenue / np.nansum(total_revenue) 

    wrmsse = np.nansum(weight * rmsse)
    
    return wrmsse

calculate_wrmsse(test_labels, test_forecasts) # 12